In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

torch.cuda.is_available()

# 定义稀疏自编码器模型
class SparseAutoencoder(nn.Module):
    def __init__(self, input_dim, encoding_dim):
        super(SparseAutoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, encoding_dim),
            nn.Sigmoid()  # 用Sigmoid激活函数产生稀疏性
        )
        self.decoder = nn.Sequential(
            nn.Linear(encoding_dim, 128),
            nn.ReLU(),
            nn.Linear(128, input_dim),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# 准备输入数据
input_dim = 163  # 输入数据维度
encoding_dim = 10  # 编码后的维度
num_samples = 1000
data = np.random.rand(num_samples, input_dim)
data = torch.from_numpy(data).float()

# 创建稀疏自编码器模型
model = SparseAutoencoder(input_dim, encoding_dim)

# 定义损失函数和优化器
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 训练稀疏自编码器
num_epochs = 100
for epoch in range(num_epochs):
    # 前向传播
    output = model(data)
    loss = criterion(output, data)
    
    # 反向传播和优化
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# 使用稀疏自编码器进行编码
encoded_data = model.encoder(data)
